In [1]:
import os
import re
import numpy as np
import pandas as pd
from nltk import word_tokenize
import textblob as txb
from nltk.corpus import stopwords
import spacy
nlp = spacy.load("en_core_web_lg")

stop_words = set(stopwords.words('english'))

In [258]:
def get_lyrics():
    print("Loading Lyrics...")
    lyrics = []
    files = os.listdir("data/lyrics/")
    idx = [int(f.split(".")[0]) for f in files]

    for f in files:
        with open("data/lyrics/" + f, "r", encoding="utf-8") as lines:
            lyrics += [" ".join(lines)]
    return np.array(lyrics), idx
lyrics, idx = get_lyrics()

Loading Lyrics...


In [263]:
sentiment = [txb.TextBlob(l).sentiment for l in lyrics]

In [264]:
stripped = [[re.sub(r"\'|\`|\’|", "", line.strip().lower()) for line in i.split("*") if line not in [" ", "", "'"]] for i in lyrics]

In [265]:
dupe_counts = np.zeros((len(stripped), 1))

for k, song in enumerate(stripped):
    if len(song) == 0:
        continue
    song_sort = sorted(song)
    for i in range(1, len(song_sort)):
        if song_sort[i] == (song_sort[i - 1]):
            dupe_counts [k] += 1
    dupe_counts[k] /= len(song_sort)

In [266]:
joined = [" ".join([w for w in " ".join(x).split(" ")]) for x in stripped]
joined

echo_counts = np.zeros((len(joined), 1))

for k, song in enumerate(joined):
    if len(song) == 0:
        continue
    song = song.split(" ")
    for i in range(1, len(song)):
        if song[i] == (song[i - 1]):
            echo_counts [k] += 1
    echo_counts[k] /= len(song)

In [267]:
spacies = [nlp(x) for x in joined]


In [268]:
futures = np.zeros((len(spacies)))

for i, s in enumerate(spacies):
    for w in range(1, len(s)):
        if (s[w - 1].dep_ == 'aux' and s[w - 1].text in ['will', 'shall'] and s[w].tag_ == 'VB'):
            futures[i] += 1

In [269]:
from collections import Counter

counts = [Counter(([token.tag_ for token in s])) for s in spacies]

presents = list(map(add, futures * (-1), np.array([c["VB"] + c["VBG"] + c["VBP"] + c["VBZ"] for c in counts])))
pasts = np.array([c["VBD"] + c["VBN"] for c in counts], dtype=float)
all_verbs = presents + pasts + futures

presents /= all_verbs
futures /= all_verbs
pasts /= all_verbs

In [270]:
adjectives = np.array([c["JJ"] + c["JJR"] + c["JJS"] for c in counts]).reshape(len(counts), 1)

In [271]:
titles = pd.read_csv("data/raw/SpotifyData.csv")[["title"]].to_numpy()[idx]
title_in_song = np.array([re.search(titles[i][0], joined[i]) != None for i in range(len(titles))]).reshape(titles.shape[0], 1)

In [283]:
puncts = np.array([(c["``"] + c["''"] + c[","] + c["."] + c[":"] + c["NFP"]) /sum(list(c.values())) for c in counts]).reshape(len(counts), 1)
puncts

array([[0.025     ],
       [0.13211382],
       [0.0443787 ],
       ...,
       [0.00829876],
       [0.00641026],
       [0.0530303 ]])

In [284]:
futures = futures.reshape((futures.shape[0], 1))
pasts = pasts.reshape((pasts.shape[0], 1))
presents = presents.reshape((presents.shape[0], 1))
from sklearn import preprocessing


export = pd.DataFrame(preprocessing.scale(np.hstack((dupe_counts, echo_counts, presents, futures, pasts, adjectives, np.array(sentiment), title_in_song, puncts))), columns=["dupe_counts", "echo_counts", "presents", "futures", "pasts", "adjectives", "sentiment", "subjectivity", "title_in_song", "puncts"])
# export = preprocessing.scale(export)

export.to_csv("sentiment.csv")

In [261]:
lyrics[0]

'[Verse 1]\n It’s late\n Time for bed\n So I sit and I wait\n For that gin and tonic\n To go to your head\n \n [Verse 2]\n I know\n It’s a devious plan\n But it’s the only way that I know\n To get those big bad car keys\n Out of your hand\n \n [Chorus]\n You know\n That I remain a gentleman\n But even so\n There’s only so much\n A gentleman can stand\n Sleep with me\n Oh\n Sleep with me, tonight\n \n [Instrumental]\n \n [Verse 3]\n My cards are on your table\n My dreams are in your bed\n Oh, if I was able\n I’d be there instead\n \n [Outro]\n Oh, oh\n Oh, sleep with me tonight\n \n [Instrumental]'

In [282]:
sum(list(counts[0].values()))

160